In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np

db=mysql.connector.connect( host='localhost',
                            user="root",
                            password="1234567890",
                            database="ecommerce")

cur=db.cursor()

# List all unique cities where customers are located.


In [21]:
query = """SELECT DISTINCT customer_city FROM customers"""
cur.execute(query)
data = cur.fetchall()
data

[('franca',),
 ('sao bernardo do campo',),
 ('sao paulo',),
 ('mogi das cruzes',),
 ('campinas',),
 ('jaragua do sul',),
 ('timoteo',),
 ('curitiba',),
 ('belo horizonte',),
 ('montes claros',),
 ('rio de janeiro',),
 ('lencois paulista',),
 ('caxias do sul',),
 ('piracicaba',),
 ('guarulhos',),
 ('pacaja',),
 ('florianopolis',),
 ('aparecida de goiania',),
 ('santo andre',),
 ('goiania',),
 ('cachoeiro de itapemirim',),
 ('sao jose dos campos',),
 ('sao roque',),
 ('camacari',),
 ('resende',),
 ('sumare',),
 ('novo hamburgo',),
 ('sao luis',),
 ('sao jose',),
 ('santa barbara',),
 ('ribeirao preto',),
 ('ituiutaba',),
 ('taquarituba',),
 ('sao jose dos pinhais',),
 ('barrinha',),
 ('parati',),
 ('dourados',),
 ('trindade',),
 ('cascavel',),
 ('fortaleza',),
 ('brasilia',),
 ('pelotas',),
 ('porto alegre',),
 ('salto',),
 ('jundiai',),
 ('cacapava',),
 ('sao vicente',),
 ('uberlandia',),
 ('botelhos',),
 ('sao goncalo',),
 ('araucaria',),
 ('nova iguacu',),
 ('areia branca',),
 ('campo

 # Count the number of orders placed in 2017.

In [22]:
query = """select count(order_id) from orders where year(order_purchase_timestamp)=2017"""
cur.execute(query)
data = cur.fetchall()
data[0][0]

90202

 # Find the total sales per category.

In [ ]:
query = """select upper(products.product_category )as category, round(sum(payments.payment_value),2) as sales 
from products join order_items on products.product_id= order_items.product_id
join payments on payments.order_id = order_items.order_id
group by category;"""
cur.execute(query)
data = cur.fetchall()
data
df=pd.DataFrame(data,columns=["category","sales"])
df


# Calculate the percentage of orders that were paid in installments

In [ ]:
query="""select sum(case 
when payment_installments>1 then 1 
else 0
end)/count(*) * 100
 from payments"""
cur.execute(query)
data = cur.fetchall()
data[0][0]

 # Count the number of customers from each state. 

In [ ]:
query="""select customer_state,count(customer_id) from customers group by customer_state"""
cur.execute(query)
data = cur.fetchall()
data
df=pd.DataFrame(data,columns=["state","Number_of_customers"])
df

plt.figure(figsize=(9,5))
plt.bar(df["state"],df["Number_of_customers"])

# Calculate the number of orders per month in 2018.

In [ ]:
query="""select monthname(order_purchase_timestamp) months, count(order_id)  orders from orders 
where year(order_purchase_timestamp)=2018
 group by months order by months"""
cur.execute(query)
data = cur.fetchall()
data
df=pd.DataFrame(data,columns=["months","orders"])
df


#  Find the average number of products per order, grouped by customer city.


In [ ]:
query="""with order_count_table as
(select orders.order_id,orders.customer_id, count(order_items.order_id) as OC
from orders join order_items
on orders.order_id=order_items.order_id
group by orders.order_id,orders.customer_id)

select customers.customer_city, round(avg(order_count_table.OC),2)
from customers join order_count_table
on customers.customer_id=order_count_table.customer_id
group by customers.customer_city order by avg(order_count_table.OC) desc
"""
cur.execute(query)
data = cur.fetchall()
data
df=pd.DataFrame(data)
df

# Calculate the percentage of total revenue contributed by each product category.


In [ ]:
query="""select upper(products.product_category )as category, 
round((sum(payments.payment_value)/(select sum(payment_value)from payments))*100,2) as sales
from products join order_items on products.product_id= order_items.product_id
join payments on payments.order_id = order_items.order_id
group by category;
select sum(payment_value)from payments order by sales desc"""
cur.execute(query)
data=cur.fetchall()
data
df=pd.DataFrame(data,columns=["category","percent_sales"])
df

#  Identify the correlation between product price and the number of times a product has been purchased.


In [ ]:
query="""Select products.product_category,count(order_items.product_id),ROUND(avg(order_items.price),2)
from products join order_items
on products.product_id=order_items.product_id
group by products.product_category"""

cur.execute(query)
data=cur.fetchall()
data
df=pd.DataFrame(data,columns=["category","Orders","price"])

arry1 = df["Orders"].astype(float)
arry2 = df["price"].astype(float)

correlation_matrix = np.corrcoef(arry1, arry2)
print(correlation_matrix)

# Calculate the total revenue generated by each seller, and rank them by revenue.


In [ ]:
query="""select *, dense_rank() over(order by revenue desc) as seller_rank from 
(SELECT order_items.seller_id, round(sum(payments.payment_value),2) as revenue from order_items join payments
on order_items.order_id=payments.order_id
group by order_items.seller_id) as a"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Seller_id","Revenue","Rank"])
df.head(10)
plt.figure(x="Seller_id",y="Revenue")
plt.xticks(rotation=90)
plt.show()

# Calculate the moving average of order values for each customer over their order history.


In [ ]:
query="""select customer_id, order_purchase_timestamp,payment_value,
avg(payment_value) over(partition by customer_id order by order_purchase_timestamp
rows between 2 preceding and current row) as mov_avg
from
(select orders.customer_id, orders.order_purchase_timestamp,payments.payment_value
from orders join payments
on orders.order_id=Payments.order_id) as a;
select orders.customer_id, orders.order_purchase_timestamp,payments.payment_value
from orders join payments
on orders.order_id=Payments.order_id"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["customer_id","order_time","sale","mov_avg"])
df.head(10)

#  Calculate the cumulative sales per month for each year.


In [41]:
query="""select years, months, payments, sum(payments) 
over(order by years,months)from 
(select year(orders.order_purchase_timestamp)as years,month(orders.order_purchase_timestamp) as months,
payments.payment_value as payments
from orders join payments
on orders.order_id=payments.order_id) as a"""
cur.execute(query)
data=cur.fetchall()
data

[(2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 75.06, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 40.95, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 9, 136.23, 1513.4399642944336),
 (2016, 10, 149.91, 356056.31996011734),
 (2016, 10, 149.91, 356056.31996011734),
 (2016, 10, 149.91, 356056.31996011734),
 (2016, 10, 149.91, 356056.31996011734),
 (2016, 10, 317.9, 356056.31996011734),
 (2016, 10, 317.9, 356056.31996011734),
 (2016, 10, 34.42, 356056.31996011734),
 (2016, 